In [21]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import pytz

from post_processing.dataclass.data_aplose import DataAplose
from post_processing.utils.fpod_utils import cpod2aplose, usable_data_phase, dpm_to_dph,build_hour_range, fb_folder,csv_folder, meta_cut_aplose
from post_processing.utils.core_utils import json2df

### Pre-processing



In [22]:
data = csv_folder(r"U:\A")
print(data.head())

df_1 = data.dropna()

df_aplose = cpod2aplose(df_1, pytz.utc, "Site A Haute", "Commerson", extra_columns=["DPM", "deploy.name"])
print(df_aplose.head())

                                   File          ChunkEnd  DPM  Nall  MinsOn  \
0  Ile Haute Site A  POD3059 file01.CP3  01/12/2022 16:03    0     0       0   
1  Ile Haute Site A  POD3059 file01.CP3  01/12/2022 16:04    0     0       0   
2  Ile Haute Site A  POD3059 file01.CP3  01/12/2022 16:05    0     0       0   
3  Ile Haute Site A  POD3059 file01.CP3  01/12/2022 16:06    0     0       0   
4  Ile Haute Site A  POD3059 file01.CP3  01/12/2022 16:07    0     0       0   

            deploy.name  
0  Site A Haute_Phase11  
1  Site A Haute_Phase11  
2  Site A Haute_Phase11  
3  Site A Haute_Phase11  
4  Site A Haute_Phase11  
        dataset filename  start_time  end_time  start_frequency  \
0  Site A Haute                    0        60                0   
1  Site A Haute                    0        60                0   
2  Site A Haute                    0        60                0   
3  Site A Haute                    0        60                0   
4  Site A Haute             

In [23]:
json = Path(r"C:\Users\fouinel\Downloads\deployment_kerguelen.json")
metadatax = json2df(json_path=json)

metadatax["deploy.name"] = (metadatax["site.name"].astype(str) + "_" +
                           metadatax["campaign.name"].astype(str))

cleared = meta_cut_aplose(df_aplose, metadatax)

cleared.to_csv(r"U:\APLOSE_A.csv", index=False)

## Load data

In [ ]:
yaml_file = Path(r"resource\CPOD-FPOD_yaml.yml")
data_list = DataAplose.from_yaml(file=yaml_file)
print(data_list.df.head())

## Create a detection per hour dataframe



In [ ]:
dph = dpm_to_dph(data_list.df, extra_columns=["deploy.name"])

## Add the feeding buzzes

In [ ]:
fb = fb_folder(r"U:\fb_A_NBHF", "Commerson")
fb["Date heure"] = fb["start_datetime"].dt.floor("h")
fb = fb.groupby("Date heure")["Foraging"].sum().reset_index()

d_hour = dph.merge(fb[["Date heure","Foraging"]], on="Date heure", how="left")

In [ ]:
d_hour[["DPM","Foraging"]] = d_hour[["DPM","Foraging"]].fillna(0)
print(d_hour.head())

In [ ]:
d_hour["Year"] = d_hour["Date heure"].dt.year
d_hour["Month"] = d_hour["Date heure"].dt.month
d_hour["Day"] = d_hour["Date heure"].dt.day
d_hour["Hour"] = d_hour["Date heure"].dt.hour

d_hour["FBR"] = d_hour["Foraging"] / d_hour["DPM"]

In [ ]:
d_hour["FBR"] = d_hour["FBR"].fillna(0)
d_hour.to_csv(r"U:\Hours_DPM_FBUZZ_SiteA.csv", index=False)